# Filtrage passe-bas, à bande étroite et analyse de la TZ

*But* : Analyser les filtres à bandes étroites et comprendre comment les représenter via la TZ



## 1. Filtre passe-haut

- On commence par charger le fichier du notebook précédent

In [ ]:
%matplotlib inline
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

import soundfile as sf
import IPython.display 


x_loop, Fe = sf.read("myloop.wav")
x_loop = x_loop[:,0] #prenons qu'un seul canal 
IPython.display.Audio(x_loop, rate=Fe)



- Un filtre passe-haut amplifie, comme son nom l'indique, les hautes fréquences jusqu'à une certaine valeur.  

Nous allons d'abord étudier un filtre de fonction de transfert suivante : 

$$
H(z) = \frac{1-z^{-1}}{1+0.5z^{-1}}
$$

**Questions et application**:
1. Vérifier ses caractéristiques (stable, causal etc. ?)
2. Déterminer son pôle et son zéro
3. Tracer la réponse en fréquence du filtre ainsi que les pôles et zéros sur un cercle (aidez vous des fonctions ci-dessous)
4. Commentez ce que vous entendez.

In [ ]:
def reponse_en_frequence(b, a):
    w,h = sig.freqz(b,a,4096)
    hdb = 20*np.log10(np.abs(h))
    plt.plot(w/2/np.pi,hdb)
    plt.grid()
    plt.xlabel(r'fréquence réduite $\nu$')
    plt.ylabel('dB')
    plt.title('Réponse en fréquence du filtre')
    plt.show()



def plot_poles_and_zeros(poles, zeros):
    # Plot the poles and zeros
    plt.plot(np.real(zeros), np.imag(zeros), 'o', label='Zeros')
    plt.plot(np.real(poles), np.imag(poles), 'x', label='Poles')

    # Add axis labels and a legend
    plt.xlabel('Partie réelle')
    plt.ylabel('Partie Imaginaire')
    plt.title('Pôles et zéros')
    plt.legend()


    # Ajouter un cercle unité
    theta = np.linspace(0, 2*np.pi, 100)
    plt.plot(np.cos(theta), np.sin(theta), '--', color='gray')

    plt.xlim([-2.5, 2.5])
    plt.ylim([-2.5, 2.5])
    plt.gca().set_aspect('equal', adjustable='box')

    #afficher le plot
    plt.grid()
    plt.show()


In [ ]:
a = TODO #  dénominateur
b = TODO #  numérateur

x_f = sig.lfilter(b,a,x_loop)

pi = np.pi


# Calculer les pôles et les zéros
zeros = np.roots(b)
poles = np.roots(a)

# Si on part des poles et des zéros, on a 
# a = np.poly(zeros)
# b = np.poly(poles)

reponse_en_frequence(b, a)

plot_poles_and_zeros(poles, zeros)

IPython.display.Audio(x_f, rate=Fe)


## 2. Filtre passe-bas

1. Commencez par inverser le dénominateur et le numérateur du filtre précédent. C'est à dire, considérez la fonction de transfert suivante : 
$$
H(z) = \frac{1+0.5z^{-1}}{1-z^{-1}}
$$
Faites la même analyse qu'avant (réponse en fréquence, poles/zéros, stabilité, causalité etc.)

2. Vous recevez un message d'erreur ? Quelle propriété n'a pas ce filtre qu'avait l'autre filtre ? 
   
3. On va remplacer par le filtre suivant : 
$$
H(z) = \frac{1+0.5z^{-1}}{1-0.95z^{-1}}
$$
Est-ce que la propriété du premier filtre (dans la section 1) manquante a été récupérée ? 

4. Essayez de jouer avec les valeurs du pôle et du zéro pour voir l'influence sur la réponse en fréquence. En conclure alors à partir de cette première section et de la seconde section les valeurs des points $[1, 0], [-1, 0]$ sur le cercle (au choix: $\frac{fe}{2} Hz$ ou $0$) 

# 3. Filtre Passe-bande
## 3.1. L'effet "vieux téléphone"
Les appels téléphoniques, pour des raisons techniques, étaient limités à une bande fréquentielle de $200 Hz – 3,4 kHz$ dans le passé. 

On va charger tout d'abord le fichier ``speech.wav`` et tenter d'appliquer un filtre passe-bande correspondant à $200HZ - 3.4 kHz$

In [ ]:
x_speech, Fe = sf.read("speech.wav")
print(Fe)
IPython.display.Audio(x_speech, rate=Fe)

1. Déterminer les angles $\theta_{\text{in}}, \theta_{\text{out}}$ sur le demi-cercle unité correspondant aux fréquences $200Hz$ et $3.4 kHz$ respectivement. 
2. Soient $z_{\text{in}} = re^{i\theta_{\text{in}}}$ et $z_{\text{out}} = re^{i\theta_{\text{out}}}$ les pôles correspondant aux fréquences du filtre passe bande avec $r < 1$. On considère $1$ pôle $z_1$ son conjugué :
$$
H(z) = \frac{1}{|1-z_1 z^{-1}|^2}
$$
Pourquoi considérer les pôles conjugués à votre avis ?

1. Application : Prenez $1$ pôle entre $\theta_{\text{in}}$ et $\theta_{\text{out}}$ au centre de la bande et appliquez-le au filtre. Observez la réponse fréquentielle du filtre en fonction des paramètres, contrôlez la position des pôles et concluez.
   
2. Amélioration : Rajoutez des zéros conjugués dans $H(z)$ pour encore plus insister sur la réponse fréquentielle du filtre dans la bande passante voulue.  

In [ ]:
r_zeros = TODO
r_pole = TODO
Theta_in = TODO
Theta_out = TODO
thetas_poles = TODO
thetas_zeros = TODO

z_pole = r_pole * np.concatenate((np.exp(1j * thetas_poles), (np.exp(-1j * thetas_poles))),
                       axis=0)
z_zeros = r_zeros * np.concatenate((np.exp(1j * thetas_zeros), (np.exp(-1j * thetas_zeros))),
                       axis=0)
a = np.poly(z_pole)  # denominateur
b = np.poly(z_zeros) # numérateur

x_f = TODO

reponse_en_frequence(b, a)

plot_poles_and_zeros(poles, zeros)



## 3.2. Isolation des notes

Un modèle simple pour un instrument harmonique quand il joue une note est qu'il est composé : 
- d'une fréquence fondamentale (souvent appellée $f_0$)
- d'harmoniques ou *partiel harmonique* (proportionnel à $f_0$ *e.g.* $2 f_0, 3f_0 \dots$)

Les fréquences fondamentales du piano sont disponibles sur Wikipedia [ici](https://fr.wikipedia.org/wiki/Fr%C3%A9quences_des_touches_du_piano)
Le la est à 440 Hz. Quand la fréquence double, la note porte le même nom mais on change d'octave. Le rapport de fréquence entre deux notes successives est égal à 2**(1/12).

On s'intéresse tout d'abord à un fichier généré avec des sinusoïdes. Nous allons créer un filtre  stable et causal qui résonne quand une note donnée est présente dans le signal mais pas pour les autres notes. On pourra 
- soit placer les pôles et les zéros de manière à obtenir l'effet voulu, 
- soit partir d'un filtre passe-bas et décaler sa réponse en fréquence de manière appropriée.


Conseil: Il peut être intéressant de retourner le module d'un filtre complexe. En effet la sortie d'un filtre réel oscillera à la fréquence recherchée, ce qui gêne la visualisation.

In [ ]:
t = np.arange(0, 5, 1/Fe)
son = np.concatenate([np.sin(2 * np.pi * t[0:len(t)//3] * 440), 
                      np.sin(2*np.pi*t[len(t)//3:2*len(t)//3] * 440*2**(1/12)), 
                      np.sin(2*np.pi*t[2*len(t)//3:] * 440 * 2) ])

In [ ]:
IPython.display.Audio(son, rate=Fe)

On charge ci-après le fichier ``Piano_accord.wav`` qui est composé des notes $\texttt{Do 4, Mi 4, Sol 4} $. Essayez de faire de même avec ce morceau plus complexe.

In [ ]:
x_accord, Fe = sf.read("Piano_accord.wav")
x_accord = x_accord[:,0] #prenons qu'un seul canal 
IPython.display.Audio(x_accord, rate=Fe)

## 3.3. Transformée de Fourier à court terme
Montrez que la TCFT peut se réinterpréter comme un ensemble de filtres détecteurs de notes. En déduire un algorithme rapide pour détecter un grand nombre de notes en même temps.
